In [ ]:
# %% [code] Install dependencies with precise versions
!pip install -q -U \
    transformers==4.40.0 \
    datasets==2.18.0 \
    accelerate==0.29.2 \
    peft==0.10.0 \
    bitsandbytes==0.43.0 \
    trl==0.8.0 \
    sentencepiece==0.1.99 \
    sacrebleu==2.3.1 \
    comet-ml==3.35.0 \
    tensorboardX==2.6.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.1/580.1 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.


In [ ]:
# %% [code] Imports
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
import numpy as np
from huggingface_hub import notebook_login, HfApi
import gc
import os

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
# Evaluation metrics
from sacrebleu import corpus_bleu, CHRF
# from comet import download_model, load_from_checkpoint

In [ ]:
# %% [code] Dataset loading
def load_itihasa_dataset():
    dataset = load_dataset("rahular/itihasa")
    print(f"Train samples: {len(dataset['train'])}")
    print(f"Validation samples: {len(dataset['validation'])}")
    print(f"Test samples: {len(dataset['test'])}")
    return dataset

dataset = load_itihasa_dataset()



Train samples: 75162
Validation samples: 6149
Test samples: 11722


In [ ]:
#login

In [ ]:
# %% [code] Model setup with QLoRA
model_name = "meta-llama/Meta-Llama-3-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# %% [code] Optimized LoRA config
peft_config = LoraConfig(
    r=32,  # Reduced from 64
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # Only critical layers
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
# %% [code] Training arguments optimized for speed
training_arguments = TrainingArguments(
    output_dir="./llama3-itihasa",
    per_device_train_batch_size=4,  # Increased batch size
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Reduced accumulation
    optim="paged_adamw_8bit",
    learning_rate=1e-3,
    lr_scheduler_type="linear",
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    max_grad_norm=0.5,
    warmup_ratio=0.1,
    logging_steps=50,
    save_strategy="steps",
    save_steps=1000,
    eval_steps=500,
    evaluation_strategy="steps",
    report_to="tensorboard",
    gradient_checkpointing=True,
    group_by_length=True,
    push_to_hub=True,
    hub_model_id="sarveshchaudhari/llama3-8b-itihasa-optimized",
    load_best_model_at_end=True
)

In [ ]:
# %% [code] Corrected Dataset Handling
def format_translation(examples):
    # Ensure we return a list of strings, not dictionaries
    texts = []
    for trans in examples['translation']:
        texts.append(
            f"Translate Sanskrit to English:\n{trans['sn']}\nEnglish: {trans['en']}"
        )
    return {"text": texts}

# Apply formatting correctly
train_dataset = dataset["train"].map(
    format_translation,
    batched=True,
    remove_columns=["translation"]  # Remove original columns
)
val_dataset = dataset["validation"].map(
    format_translation,
    batched=True,
    remove_columns=["translation"]
)

Map:   0%|          | 0/75162 [00:00<?, ? examples/s]

Map:   0%|          | 0/6149 [00:00<?, ? examples/s]

In [ ]:
# %% [code] Revised Trainer Configuration
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
    # Critical formatting adjustment
    formatting_func=lambda examples: examples["text"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# %% [code] Train with memory cleanup
print(f"Total trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

print("Starting training...")
trainer.train()
print("Training complete!")

# Cleanup VRAM
del trainer
gc.collect()
torch.cuda.empty_cache()


Total trainable params: 13,631,488
Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
500,1.981900,1.974120
1000,1.890300,1.919243
1500,1.849100,1.876195
2000,1.810600,1.836914
2500,1.718300,1.808461
3000,1.707300,1.777060
3500,1.673700,1.749729
4000,1.658400,1.729505


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `

Training complete!


In [ ]:
# %% [code] Corrected Save & Push
from huggingface_hub import create_repo, HfApi, notebook_login

# 1. Authenticate properly
notebook_login()

In [ ]:
# 2. Create repository first
repo_id = "sarch7040/llama3-8b-itihasa-optimized"
create_repo(
    repo_id=repo_id,
    repo_type="model",
    private=True,  # Set to False if public
    exist_ok=True,
    token=True  # Use your access token
)

# 3. Save model
model.save_pretrained("./llama3-itihasa")
tokenizer.save_pretrained("./llama3-itihasa")

# 4. Upload with progress
api = HfApi()
api.upload_folder(
    folder_path="./llama3-itihasa",
    repo_id=repo_id,
    commit_message="Optimized fine-tuned model",
    commit_description="Llama 3 8B fine-tuned on Itihasa dataset",
    token=True
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 403 Client Error. (Request ID: Root=1-680a74a0-3e34da8c6149ae2c1fc32a53;9f5c86c2-2713-4b1f-b78d-d105ba92e1f0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B to ask for access. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B - will assume that the vocabulary was not modified.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/27.5M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

...

events.out.tfevents.1745476365.578579d8d773.33255.0:   0%|          | 0.00/6.38k [00:00<?, ?B/s]

events.out.tfevents.1745476921.578579d8d773.36032.0:   0%|          | 0.00/6.06k [00:00<?, ?B/s]

events.out.tfevents.1745478414.578579d8d773.38709.0:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sarch7040/llama3-8b-itihasa-optimized/commit/5b61575b255287788640113458f18f032303f87c', commit_message='Optimized fine-tuned model', commit_description='Llama 3 8B fine-tuned on Itihasa dataset', oid='5b61575b255287788640113458f18f032303f87c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sarch7040/llama3-8b-itihasa-optimized', endpoint='https://huggingface.co', repo_type='model', repo_id='sarch7040/llama3-8b-itihasa-optimized'), pr_revision=None, pr_num=None)

In [ ]:
# %% [code] Evaluation with METEOR and TER
!pip install -q nltk pyter3
import nltk
from nltk.translate.meteor_score import meteor_score
import pyter

nltk.download('wordnet')

def generate_translation(model, tokenizer, input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("English:")[-1].strip()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# %% [code] Corrected Model Loading
import os
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

# Create offload directory
os.makedirs("./offload", exist_ok=True)

# Load base model with proper offloading
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    offload_folder="./offload",
    offload_state_dict=True,
    low_cpu_mem_usage=True
)

# Load PEFT adapter with matching offload config
model = PeftModel.from_pretrained(
    base_model,
    "./llama3-itihasa",
    device_map="auto",
    offload_folder="./offload",
    torch_dtype=torch.bfloat16
)
model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:
# Prepare test data
test_data = dataset["test"].map(format_translation)

# Run evaluation
predictions = []
references = []

for idx, example in enumerate(test_data.select(range(1000))):  # Use enumerate for proper error reporting
    try:
        input_text = example['text'].split("English:")[0]
        ref_text = example['text'].split("English:")[1].strip()

        pred = generate_translation(model, tokenizer, input_text)
        predictions.append(pred)
        references.append(ref_text)
    except Exception as e:
        print(f"Error on example {idx}: {str(e)}")

Map:   0%|          | 0/11722 [00:00<?, ? examples/s]

TypeError: string indices must be integers

In [ ]:
# Calculate metrics
chrf = CHRF(word_order=2)
bleu = corpus_bleu(predictions, [references]).score
meteor_scores = []
ter_scores = []

for pred, ref in zip(predictions, references):
    # METEOR
    meteor_scores.append(meteor_score([ref.split()], pred.split()))

    # TER
    ter_scores.append(pyter.ter(pred.split(), ref.split()))

metrics = {
    "bleu": bleu,
    "chrf++": chrf.corpus_score(predictions, [references]).score * 100,
    "meteor": np.mean(meteor_scores) * 100,
    "ter": np.mean(ter_scores) * 100
}

print(f"BLEU: {metrics['bleu']:.2f}")
print(f"chrF++: {metrics['chrf++']:.2f}")
print(f"METEOR: {metrics['meteor']:.2f}")
print(f"TER: {metrics['ter']:.2f}")

NameError: name 'predictions' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

def load_model_no_offloading(adapter_path, base_model_name="meta-llama/Meta-Llama-3-8B"):
    """Load model with proper 4-bit quantization configuration"""
    # Configure quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    # Load tokenizer with same special tokens
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    tokenizer.pad_token = tokenizer.eos_token

    # Load base model with 4-bit config only
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # Load PEFT adapter
    model = PeftModel.from_pretrained(
        base_model,
        adapter_path,
        device_map="auto"
    )

    # Resize embeddings if special tokens were added
    model.resize_token_embeddings(len(tokenizer))

    return model, tokenizer

# Usage
model, tokenizer = load_model_no_offloading("./llama3-itihasa")

# Test inference
input_text = "सर्वे मानवाः स्वतन्त्राः समुत्पन्नाः वर्तन्ते।"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
